认识一个库的两种方法

In [ ]:
import torch
print(dir(torch.cuda.is_available))
## 双下划线是一种规范，说明变量不能篡改

print(help(torch.cuda.is_available))
## 这个命令会打开vim
## 不能加括号

dataset 和 dataloader

dataset提供一种方式获取数据及其label

dataloader打包数据：如何获取每个数据及其label，知道总共有多少数据

In [4]:
from torch.utils.data import Dataset
help(Dataset)

Help on class Dataset in module torch.utils.data.dataset:

class Dataset(typing.Generic)
 |  An abstract class representing a :class:`Dataset`.
 |  
 |  All datasets that represent a map from keys to data samples should subclass
 |  it. All subclasses should overwrite :meth:`__getitem__`, supporting fetching a
 |  data sample for a given key. Subclasses could also optionally overwrite
 |  :meth:`__len__`, which is expected to return the size of the dataset by many
 |  :class:`~torch.utils.data.Sampler` implementations and the default options
 |  of :class:`~torch.utils.data.DataLoader`.
 |  
 |  .. note::
 |    :class:`~torch.utils.data.DataLoader` by default constructs a index
 |    sampler that yields integral indices.  To make it work with a map-style
 |    dataset with non-integral indices/keys, a custom sampler must be provided.
 |  
 |  Method resolution order:
 |      Dataset
 |      typing.Generic
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __add__(self, oth

In [5]:
Dataset??

Init signature: Dataset()
Source:        
class Dataset(Generic[T_co]):
    r"""An abstract class representing a :class:`Dataset`.

    All datasets that represent a map from keys to data samples should subclass
    it. All subclasses should overwrite :meth:`__getitem__`, supporting fetching a
    data sample for a given key. Subclasses could also optionally overwrite
    :meth:`__len__`, which is expected to return the size of the dataset by many
    :class:`~torch.utils.data.Sampler` implementations and the default options
    of :class:`~torch.utils.data.DataLoader`.

    .. note::
      :class:`~torch.utils.data.DataLoader` by default constructs a index
      sampler that yields integral indices.  To make it work with a map-style
      dataset with non-integral indices/keys, a custom sampler must be provided.
    """

    def __getitem__(self, index) -> T_co:
        raise NotImplementedError

    def __add__(self, other: 'Dataset[T_co]') -> 'ConcatDataset[T_co]':
        return Co

In [9]:
from PIL import Image
img_path = "hymenoptera_data/train/ants/0013035.jpg"
img = Image.open(img_path)
img.show()

In [17]:
import os

class MyData(Dataset):      ## 继承dataset

    def __init__(self, root_dir, label_dir):        ##分别是图片地址和label地址
        ## self: 相当于指定了类中的一个全局变量
        self.root_dir = root_dir
        self.label_dir = label_dir
        self.path = os.path.join(self.root_dir, self.label_dir)
        self.img_path = os.listdir(self.path)

    def __getitem__(self, index):               ## 重载了[]运算符，返回的是两个内容
        img_name = self.img_path[index]         # 这里如果没有加self，会读字符串
        img_item_path = os.path.join(self.root_dir, self.label_dir, img_name)
        img = Image.open(img_item_path)
        label = self.label_dir
        return img, label

    def __len__(self):                          ## 重载len
        return len(self.img_path)


root = "hymenoptera_data/train"
ants_label = "ants"
bees_label = "bees"
ants_dataset = MyData(root, ants_label)
bees_dataset = MyData(root, bees_label)

## 小数据集打乱拼接
train_dataset = ants_dataset + bees_dataset

tensorboard的使用，transform的使用

In [18]:
from torch.utils.tensorboard import SummaryWriter
SummaryWriter??

Init signature:
SummaryWriter(
    log_dir=None,
    comment='',
    purge_step=None,
    max_queue=10,
    flush_secs=120,
    filename_suffix='',
)
Source:        
class SummaryWriter(object):
    """Writes entries directly to event files in the log_dir to be
    consumed by TensorBoard.

    The `SummaryWriter` class provides a high-level API to create an event file
    in a given directory and add summaries and events to it. The class updates the
    file contents asynchronously. This allows a training program to call methods
    to add data to the file directly from the training loop, without slowing down
    training.
    """

    def __init__(
        self,
        log_dir=None,
        comment="",
        purge_step=None,
        max_queue=10,
        flush_secs=120,
        filename_suffix="",
    ):
        """Creates a `SummaryWriter` that will write out events and summaries
        to the event file.

        Args:
            log_dir (string): Save directory location. Defa

In [23]:
writer = SummaryWriter("logs")

## 主要用到
## writer.add_image()

for i in range(100):
    writer.add_scalar("y=2x", 3*i, i)   ## tag: 标题，scalar_value: y-axis, global_step: x-axis
                                        ## tag 改变后，在tensorboard里会添加新的图表，上一个事件不会删除
                                        ## 可以在log里删掉事件。
                                        ## 官方建议一次新的训练建一个新的文件夹
writer.close()

在log里写图像

In [ ]:
img_path =  "test_dataset/train/ants_image/0013035.jpg" 

from PIL import Image


对图像进行变换

In [25]:
import numpy as np
img_array = np.array(img)

print(type(img_array))      ## numpy 型

image_path = "test_dataset/train/ants_image/0013035.jpg"
img_PIL = Image.open(image_path)
img_array = np.array(img_PIL)

## 可以看到image, 这里步数改变不影响之前add的image
writer.add_image("test", img_array, 1, dataformats = "HWC")


<class 'numpy.ndarray'>


transforms结构和用法
totensor resize
图片->工具->结果

In [27]:
from torchvision import transforms

## 通过transform.totensor解决两个问题
## 1. 如何使用
## 2. Tensor数据类型


## 使用
img_path = "test_dataset/train/ants_image/0013035.jpg"
img = Image.open(img_path)

# 创建一个工具，变换
tensor_trans = transforms.ToTensor()
tensor_img = tensor_trans(img)

print(tensor_img)


tensor([[[0.3137, 0.3137, 0.3137,  ..., 0.3176, 0.3098, 0.2980],
         [0.3176, 0.3176, 0.3176,  ..., 0.3176, 0.3098, 0.2980],
         [0.3216, 0.3216, 0.3216,  ..., 0.3137, 0.3098, 0.3020],
         ...,
         [0.3412, 0.3412, 0.3373,  ..., 0.1725, 0.3725, 0.3529],
         [0.3412, 0.3412, 0.3373,  ..., 0.3294, 0.3529, 0.3294],
         [0.3412, 0.3412, 0.3373,  ..., 0.3098, 0.3059, 0.3294]],

        [[0.5922, 0.5922, 0.5922,  ..., 0.5961, 0.5882, 0.5765],
         [0.5961, 0.5961, 0.5961,  ..., 0.5961, 0.5882, 0.5765],
         [0.6000, 0.6000, 0.6000,  ..., 0.5922, 0.5882, 0.5804],
         ...,
         [0.6275, 0.6275, 0.6235,  ..., 0.3608, 0.6196, 0.6157],
         [0.6275, 0.6275, 0.6235,  ..., 0.5765, 0.6275, 0.5961],
         [0.6275, 0.6275, 0.6235,  ..., 0.6275, 0.6235, 0.6314]],

        [[0.9137, 0.9137, 0.9137,  ..., 0.9176, 0.9098, 0.8980],
         [0.9176, 0.9176, 0.9176,  ..., 0.9176, 0.9098, 0.8980],
         [0.9216, 0.9216, 0.9216,  ..., 0.9137, 0.9098, 0.